In [3]:
import torch
from torch import nn, autograd
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split

In [4]:
d = 40
# define function approximation
class Net(nn.Module):
    def __init__(self, meta_lr=0.1):
        super(Net, self).__init__()
        # two affine maps
        self.fc1 = nn.Linear(1, d)  
        self.fc2 = nn.Linear(d, d)
        self.fc3 = nn.Linear(d, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
criterion = nn.MSELoss()

In [5]:
from torch.nn.utils import parameters_to_vector, vector_to_parameters
from multiprocessing import Pool
from copy import deepcopy

def jacobian(grad_flat, model):

    dim = grad_flat.size()[0]
    J = torch.zeros(dim, dim)
    ''' substitute the for loop below
    def _helper(i):
        J[i,:] = parameters_to_vector(autograd.grad(grad_flat[i], model.parameters(), retain_graph=True))
    with Pool(processes=8) as pool:
        pool.map(_helper, range(dim))
    ''' 
    for i in range(dim):
        J[i,:] = parameters_to_vector(autograd.grad(grad_flat[i], model.parameters(), retain_graph=True))
    return J.cpu().data.numpy()  
def my_solve(H, v): # solve Hw = v for w
    c = 0.9/np.linalg.eigvalsh(H).max()
    #c = 0.1#0.9/L_g#H.diagonal().max()#np.linalg.eigvalsh(H).max()#0.5/L_g
    w = np.copy(v)
    for _ in range(5):
        v = v - c * H @ v
        w += v    
    return c*w

In [6]:
def single_task_loss(model, x, y):
    return criterion(model.forward(x), y)

def regularization(model, model_meta, gamma):
    reg = 0
    for (param1, param2) in zip(model.parameters(), model_meta.parameters()):
        reg += ((param1 - param2) ** 2).sum()
    return reg*(gamma*0.5)

In [7]:
net = Net()
v = deepcopy(list(net.parameters()))
dim = sum([param.numel() for param in net.parameters()])
v1 = np.random.randn(dim)
vector_to_parameters(torch.from_numpy(v1), v)

In [21]:
class STABLE(object):
    def __init__(self, ntask, device='cpu', alpha=1e-3, beta=1e-2, tau=0.1, gamma = 0.1):
        self.name = 'STABLE'
        self.ntask = ntask
        self.device = device
        self.gamma = torch.tensor(gamma).to(device)
        self.alpha = torch.tensor(alpha).to(device)
        self.beta = torch.tensor(beta).to(device)
        self.tau = tau
        self.H = {t: None for t in range(self.ntask)}
        self.v = None
        self.v_flat = 0
        self.Hv = {t: None for t in range(self.ntask)}
    
    def inner_update_single(self, model, model_meta, x, y, t):
        x, y = x.to(self.device), y.to(self.device)
        loss = single_task_loss(model, x, y) + regularization(model, model_meta, self.gamma)
        grad = autograd.grad(loss, model.parameters(), create_graph=False)
        vector_to_parameters(torch.from_numpy(my_solve(self.H[t], self.v_flat)), self.Hv[t])
        for (param, g, h) in zip(model.parameters(), grad, self.Hv[t]):
            param.data.add_(g.data, alpha=-self.beta)
            param.data.add_(h.data, alpha=-self.gamma)
        return

    def inner_update(self, models, model_meta, xlist, ylist):
        for t in range(self.ntask):
            self.inner_update_single(models[t], model_meta, xlist[t], ylist[t],t)
        return
    
    def H_update_single(self, model, model_p, model_meta, x, y, t):
        x, y = x.to(self.device), y.to(self.device)
        if not self.Hv[t]: # first iteration
            self.Hv[t] = deepcopy(list(model.parameters()))         
            dim = sum([param.numel() for param in model.parameters()]) 
            self.H[t] = np.eye(dim)*10
            return
        # estimate H given current model, model_meta
        loss = single_task_loss(model, x, y) + regularization(model, model_meta, self.gamma)
        grad = autograd.grad(loss, model.parameters(), create_graph=True)
        grad_flat = parameters_to_vector(grad)
        H = jacobian(grad_flat, model) # cpu, numpy
        # estimate H_p given previous model_p, model_meta_p
        loss = single_task_loss(model_p, x, y) + regularization(model_p, model_meta, self.gamma) # should use model_meta_p, but no difference
        grad = autograd.grad(loss, model_p.parameters(), create_graph=True)
        grad_flat = parameters_to_vector(grad)
        H_p = jacobian(grad_flat, model_p) # cpu, numpy
        self.H[t] = (1 - self.tau) * (self.H[t] - H_p) + H
        return
    
    def H_update(self, models, models_p, model_meta, xlist, ylist):
        for t in range(self.ntask):
            self.H_update_single(models[t], models_p[t], model_meta, xlist[t], ylist[t], t)
        return
    
    def meta_grad_single(self, model, x, y, t):
        x, y = x.to(self.device), y.to(self.device)
        loss = single_task_loss(model, x, y)
        grad = autograd.grad(loss, model.parameters(), create_graph=False)
        grad_flat = parameters_to_vector(grad).cpu().data.numpy()
        meta_grad_flat = my_solve(self.H[t], grad_flat) # * (- self.gamma)
        return meta_grad_flat
    
    def meta_grad(self, models, xlist, ylist):
        mg_flat_sum = 0
        
        ''' use pool to speed up the for loop
        with Pool(processes=4) as pool:
            mg_flat = pool.starmap(self.meta_grad, [(model_list[t], xlist[t], ylist[t], t) for t in range(self.ntask)])
        mg_flat = np.vstack(mg_flat)
        mg_flat = np.sum(mg_flat, dim = 1)
        '''
        
        for t in range(self.ntask):
            mg_flat = self.meta_grad_single(models[t], xlist[t], ylist[t], t)
            mg_flat_sum = mg_flat_sum + mg_flat
        
        # update self.v = new meta params - prev meta paramas = - lr * meta_gradient
        if not self.v: self.v = deepcopy(list(models[0].parameters()))
        mg_flat_sum = torch.from_numpy(mg_flat_sum).to(self.device)
        self.v_flat = mg_flat_sum * self.gamma * (-self.alpha)
        vector_to_parameters(self.v_flat, self.v)
        self.v_flat = self.v_flat.cpu().data.numpy()
          
    def meta_update(self, model_meta):       
        # update meta params
        for (param, v_item) in zip(model_meta.parameters(), self.v):
            param.data.add(v_item.data)      

In [22]:
class TTSA(object):
    def __init__(self, ntask, device='cpu', alpha=1e-3, beta=1e-2, gamma = 0.1):
        self.name = 'TTSA'
        self.ntask = ntask
        self.device = device
        self.gamma = torch.tensor(gamma).to(device)
        self.alpha = torch.tensor(alpha).to(device)
        self.beta = torch.tensor(beta).to(device)
        self.iter = 0
        self.L = 10
        self.batch_size = 10
    
    def inner_update_single(self, model, model_meta, x, y, t):
        x, y = x.to(self.device), y.to(self.device)
        loss = single_task_loss(model, x, y) + regularization(model, model_meta, self.gamma)
        grad = autograd.grad(loss, model.parameters(), create_graph=False)
        for (param, g) in zip(model.parameters(), grad):
            param.data.add_(g.data, alpha=-self.beta)
        return
    
    def inner_update(self, models, model_meta, xlist, ylist):
        for t in range(self.ntask):
            self.inner_update_single(models[t], model_meta, xlist[t], ylist[t],t)
        return
    
    def meta_grad_single(self, model, x, y, t):
        bs = self.batch_size
        x, y = x.to(self.device), y.to(self.device)
        loss = single_task_loss(model, x[0:bs], y[0:bs])
        mg = autograd.grad(loss, model.parameters(), create_graph=False)
        
        #p1 = torch.ceil((torch.log(self.iter + 1))) + 1
        #p2 = torch.random.randint(p1) # p = len(x_list) = len(y_list)
        p1, p2 = 10, 5
        for j in range(p2):
            loss = single_task_loss(model, x[(j+1)*bs:(j+2)*bs], y[(j+1)*bs:(j+2)*bs])
            grad = autograd.grad(loss, model.parameters(), create_graph=True)
            Hmg = autograd.grad(grad, model.parameters(), grad_outputs=mg, retain_graph=True)
            for (mg_item, Hmg_item) in zip(mg, Hmg):
                mg_item.data.add_(Hmg_item.data, alpha=-1/self.L)
        for mg_item in mg:
            mg_item.data = mg_item.data * (p1/self.L)
        return mg
    
    def meta_update(self, models, meta_model, xlist, ylist):
        mg = []
        for t in range(self.ntask):
            mg.append(self.meta_grad_single(models[t], xlist[t], ylist[t], t))
        for t in range(self.ntask):
            for (param, mg_item) in zip(meta_model.parameters(), mg[t]):
                param.data.add_(mg_item.data, alpha=-self.gamma*self.alpha)
        return

In [29]:
class Meta(object):
    def __init__(self, ntask, method='STABLE', device='cpu'):
        self.ntask = ntask
        self.device = device
        self.net_meta = Net().to(self.device)
        self.nets = [deepcopy(self.net_meta).to(self.device) for t in range(self.ntask)]
        self.nets_p = [deepcopy(self.net_meta).to(self.device) for t in range(self.ntask)]
        if method == 'STABLE':
            self.method = STABLE(ntask=self.ntask, device=device)
        elif method == 'TTSA':
            self.method = TTSA(ntask=self.ntask, device=device)
        
    def meta_train(self, xlist, ylist, x1list, y1list, x2list, y2list):      
        if self.method.name == 'STABLE':
            self.method.H_update(self.nets, self.nets_p, self.net_meta, xlist, ylist)
            for t in range(self.ntask):
                self.nets_p[t].load_state_dict(self.nets[t].state_dict())
            self.method.meta_grad(self.nets, x1list, y1list)
        self.method.inner_update(self.nets, self.net_meta, x2list, y2list)
        if self.method.name == 'STABLE':
            self.method.meta_update(self.net_meta)
        elif self.method.name == 'TTSA':
            self.method.meta_update(self.nets, self.net_meta, x1list, y1list)

    def test(self, xlist, ylist):
        loss = 0
        for t in range(self.ntask):
            with torch.no_grad():
                x, y = xlist[t].to(self.device), ylist[t].to(self.device)
                loss += single_task_loss(self.nets[t], x, y)
        return loss
    
    def meta_test(self, xlist, ylist):
        loss = []
        net = deepcopy(self.net_meta)
        opt = optim.SGD(net.parameters(), lr = 1e-2)
        for (x, y) in zip(xlist, ylist):
            opt.zero_grad()
            x, y = x.to(self.device), y.to(self.device)
            loss1 = single_task_loss(net, x, y)
            loss.append(loss1.item())
            loss1 = loss1 + regularization(net, self.net_meta, 0.1)
            loss1.backward()
            opt.step()
        loss = np.array(loss)
        return loss
                 

In [26]:
# configuration
import torch

# parameters
d = 40
ntask = 20#100
ntest = 20#100
# seed
seed = 2020

# hyper parameters
niters = 20#5000
nprint = 1#100
bs = 10
bs1 = 100


In [27]:
# main_function

# configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save_path = './result_sinewave/result_stable_'+str(seed)+'.txt'
save_path_test = './result_sinewave/test_stable_'+str(seed)+'.txt'
save_path_train = './result_sinewave/train_stable_'+str(seed)+'.txt'
print(save_path)

# hyperparemthers
method='STABLE' # 'TTSA'

# set random seed
torch.manual_seed(seed)

# define sine wave parameters for each task
a = []
b = []
pi = torch.acos(torch.Tensor([-1]))
for t in range(ntask):
    a.append(torch.rand(1)*4.9+0.1)
    b.append(torch.rand(1)*pi*2)

meta_learner = Meta(ntask=ntask, method=method)

test = []
train = []

epoch = 0
#alpha1 = alpha
#beta1 = beta
for iter in range(niters):
    xlist = [torch.rand(bs, 1)*10-5 for t in range(ntask)]
    ylist = [a[t]*torch.sin(xlist[t]+b[t]) for t in range(ntask)]
    x1list = [torch.rand(1*bs,1)*10-5 for t in range(ntask)]
    y1list = [a[t]*torch.sin(x1list[t]+b[t]) for t in range(ntask)]
    x2list = [torch.rand(bs,1)*10-5 for t in range(ntask)]
    y2list = [a[t]*torch.sin(x2list[t]+b[t]) for t in range(ntask)]
    
    meta_learner.meta_train(xlist, ylist, x1list, y1list, x2list, y2list)
    
    if (iter+1)%nprint == 0:
        epoch += 1
        # test loss on seen tasks
        xlist = [torch.rand(bs1,1)*10-5 for t in range(ntask)]
        ylist = [a[t]*torch.sin(xlist[t]+b[t]) for t in range(ntask)]
        loss = meta_learner.test(xlist, ylist)
        train.append(loss)
        # meta test loss on unseen tasks
        loss = 0.
        for t in range(ntest):
            atest = torch.rand(1)*4.9+0.1
            btest = torch.rand(1)*pi*2
            xtest = [torch.rand(bs,1)*10-5 for i in range(10)]
            ytest = [atest*torch.sin(xtest[i]+btest) for i in range(10)]
            loss += meta_learner.meta_test(xtest, ytest)
        test.append(loss)
        print('epoch %d:train loss %.2f, test loss %.2f' %(epoch, train[-1], test[-1][-1]))
np.savetxt(save_path_test, test)
np.savetxt(save_path_train, train)

./result_sinewave/result_stable_2020.txt
epoch 1:train loss 89.19, test loss 75.79
epoch 2:train loss 91.38, test loss 104.61
epoch 3:train loss 85.99, test loss 73.98
epoch 4:train loss 83.06, test loss 32.91
epoch 5:train loss 84.44, test loss 95.26
epoch 6:train loss 78.61, test loss 85.98
epoch 7:train loss 83.48, test loss 49.37
epoch 8:train loss 78.36, test loss 68.14
epoch 9:train loss 79.38, test loss 74.11
epoch 10:train loss 81.54, test loss 51.19
epoch 11:train loss 77.07, test loss 63.24
epoch 12:train loss 78.42, test loss 45.45
epoch 13:train loss 81.55, test loss 73.93
epoch 14:train loss 77.17, test loss 45.84
epoch 15:train loss 76.41, test loss 80.53
epoch 16:train loss 78.41, test loss 92.72
epoch 17:train loss 78.57, test loss 48.13
epoch 18:train loss 74.71, test loss 101.65
epoch 19:train loss 76.75, test loss 70.21
epoch 20:train loss 75.77, test loss 59.87


FileNotFoundError: [Errno 2] No such file or directory: './result_sinewave/test_stable_2020.txt'

In [28]:
# main_function

# configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save_path = './result_sinewave/result_stable_'+str(seed)+'.txt'
save_path_test = './result_sinewave/test_stable_'+str(seed)+'.txt'
save_path_train = './result_sinewave/train_stable_'+str(seed)+'.txt'
print(save_path)

# hyperparemthers
method='TTSA'

# set random seed
torch.manual_seed(seed)

# define sine wave parameters for each task
a = []
b = []
pi = torch.acos(torch.Tensor([-1]))
for t in range(ntask):
    a.append(torch.rand(1)*4.9+0.1)
    b.append(torch.rand(1)*pi*2)

meta_learner = Meta(ntask=ntask, method=method)

test = []
train = []

epoch = 0
#alpha1 = alpha
#beta1 = beta
for iter in range(niters):
    xlist = [torch.rand(bs, 1)*10-5 for t in range(ntask)]
    ylist = [a[t]*torch.sin(xlist[t]+b[t]) for t in range(ntask)]
    x1list = [torch.rand(10*bs,1)*10-5 for t in range(ntask)]
    y1list = [a[t]*torch.sin(x1list[t]+b[t]) for t in range(ntask)]
    x2list = [torch.rand(bs,1)*10-5 for t in range(ntask)]
    y2list = [a[t]*torch.sin(x2list[t]+b[t]) for t in range(ntask)]
    
    meta_learner.meta_train(xlist, ylist, x1list, y1list, x2list, y2list)
    
    if (iter+1)%nprint == 0:
        epoch += 1
        # test loss on seen tasks
        xlist = [torch.rand(bs1,1)*10-5 for t in range(ntask)]
        ylist = [a[t]*torch.sin(xlist[t]+b[t]) for t in range(ntask)]
        loss = meta_learner.test(xlist, ylist)
        train.append(loss)
        # meta test loss on unseen tasks
        loss = 0.
        for t in range(ntest):
            atest = torch.rand(1)*4.9+0.1
            btest = torch.rand(1)*pi*2
            xtest = [torch.rand(bs,1)*10-5 for i in range(10)]
            ytest = [atest*torch.sin(xtest[i]+btest) for i in range(10)]
            loss += meta_learner.meta_test(xtest, ytest)
        test.append(loss)
        print('epoch %d:train loss %.2f, test loss %.2f' %(epoch, train[-1], test[-1][-1]))
np.savetxt(save_path_test, test)
np.savetxt(save_path_train, train)

./result_sinewave/result_stable_2020.txt
epoch 1:train loss 86.83, test loss 64.01
epoch 2:train loss 86.52, test loss 60.23
epoch 3:train loss 84.40, test loss 68.49
epoch 4:train loss 83.72, test loss 50.02
epoch 5:train loss 81.45, test loss 64.62
epoch 6:train loss 80.24, test loss 69.97
epoch 7:train loss 80.15, test loss 68.74
epoch 8:train loss 80.94, test loss 91.07
epoch 9:train loss 80.24, test loss 80.94
epoch 10:train loss 82.06, test loss 74.99
epoch 11:train loss 78.96, test loss 59.15
epoch 12:train loss 77.38, test loss 76.63
epoch 13:train loss 79.64, test loss 58.49
epoch 14:train loss 73.29, test loss 84.61
epoch 15:train loss 76.52, test loss 75.31
epoch 16:train loss 72.40, test loss 72.41
epoch 17:train loss 78.07, test loss 29921.63
epoch 18:train loss 73.66, test loss nan
epoch 19:train loss 74.34, test loss nan
epoch 20:train loss 73.81, test loss nan


FileNotFoundError: [Errno 2] No such file or directory: './result_sinewave/test_stable_2020.txt'

In [ ]:
def eval_hessian(loss_grad, model):
    cnt = 0
    for g in loss_grad:
        g_vector = g.contiguous().view(-1) if cnt == 0 else torch.cat([g_vector, g.contiguous().view(-1)])
        cnt = 1
    l = g_vector.size(0)
    hessian = torch.zeros(l, l)
    for idx in range(l):
        grad2rd = autograd.grad(g_vector[idx], model.parameters(), create_graph=True)
        cnt = 0
        for g in grad2rd:
            g2 = g.contiguous().view(-1) if cnt == 0 else torch.cat([g2, g.contiguous().view(-1)])
            cnt = 1
        hessian[idx] = g2
    return hessian.cpu().data.numpy()

In [132]:
A = np.random.randn(10, 10) + np.eye(10)
A = A + A.T

In [133]:
print(np.linalg.eigvalsh(A))

[-5.86469287 -3.89964007 -0.64005213 -0.04511688  1.68508328  4.17302212
  5.13760779  6.31701253  8.30852771  9.45742275]


In [152]:
def my_solve(H, v): # solve Hw = v for w
    c = 0.9/np.abs(np.linalg.eigvalsh(H)).max()
    #c = 0.1#0.9/L_g#H.diagonal().max()#np.linalg.eigvalsh(H).max()#0.5/L_g
    w = np.copy(v)
    for _ in range(5):
        v = v - c * H @ v
        w += v    
    return c*w

In [153]:
b = np.random.randn(10)
x = my_solve(A, b)
print(A@x, b)

[ 1.00264689  1.13322862  0.30810938  1.50838238  0.88373575 -0.51292421
  0.61018006  1.60135782 -1.87168432 -1.08426956] [ 0.41786804 -0.40745863  1.07898654 -0.15645316 -0.2032048  -0.00859399
  0.31449628 -0.18460478 -0.16273575 -1.46947519]
